In [11]:
# import libraries for class
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import time

# sklearn models
from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, TweedieRegressor
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor 
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

In [23]:
class full_regression:
    """A class which automatically does many regression models and gridsearches for you. 
    Note: when you run a new model it will overwrite the previous model. You can access the current model with .model and .model_des.
    Other options:
    run_all = default True, if set to false the class will not automatically run any models
    standardize = default True, uses standard scaler and fit-transforms on train, transform on test if exists
    test_size = default 0.15, decide the side of your test set - set to 0 if dont want test
    folds = default 6, amount of folds for cross validation - integer > 1
    shuffle = default True, shuffle the data for test split and cross val
    stratify = default None, input the variable that you which to stratify the data by
    print_info = default True, print all of the results out every time you run a model
    save_it = default False, this adds functionality to be able to save down all model results into a
              dataframe, set as a global variable called model_tracker.
    comment = default None, This is a comment field for the model_tracker
    Go to readme for further information: https://github.com/LukeBetham/machine-learning-classes/blob/master/README.md
    Created by LukeBetham"""

    def __init__(self, X, y, run_all=True, standardize=True,
                 test_size=0.15, folds=6, shuffle=True, stratify=None,
                 print_info=True, save_it=False, comment=None):
        # Save settings to object
        self.folds = folds
        self.shuffle = shuffle
        self.stratify= stratify
        self.comment = comment
        self.save_it = save_it
        self.print_info = print_info
        if self.stratify is None:
            self.kfold = KFold(self.folds, shuffle=self.shuffle, random_state=66)
        else:
            self.kfold = StratifiedKFold(self.folds, shuffle=self.shuffle, random_state=66)
        # Option for bolding print text
        self.BOLD = '\033[1m'
        self.END = '\033[0m'
        # Create train-test split if selected
        self.X = X
        self.y = y
        self.test = test_size
        if self.test != 0:
            self.X, self.X_test, self.y, self.y_test, self.index_train, self.index_test = train_test_split(
                self.X, self.y, self.X.index, test_size=self.test, shuffle=self.shuffle, 
                stratify=self.stratify, random_state=66)
        # Standardise the data if selected
        if standardize == True:
            scaler = StandardScaler()
            self.X = pd.DataFrame(
                scaler.fit_transform(self.X), columns=self.X.columns)
            if self.test != 0:
                self.X_test = pd.DataFrame(
                    scaler.transform(self.X_test), columns=self.X.columns)
        # Run all models
        if run_all==True:
            self.Linear()
            self.Linear_elastic_net()
            self.Linear_generalised()
            self.SVM_regressor()
            self.KNN_regressor()
            self.Decision_tree_model()
            self.Random_forest_model()
            self.GradientBoosting()
            self.VotingRegressor()
            self.MLP_Neural_Net()

    def Linear(self, model=LinearRegression()):
        # Set up linear model
        self.model = model
        self.model_des = "Linear Regression Model"
        self.grid_multiple = 1
        self.model_calc()
        
    def Linear_elastic_net(self, model=ElasticNet()):
        # Set up linear model
        self.model = model
        self.model_des = "Linear Regression Model (with Elastic Net)"
        self.grid_multiple = 1
        self.model_calc()

    def Linear_generalised(self, model=TweedieRegressor()):
        # Set up linear model generalised - power=0: Normal, power = 1: Poisson, power = 2: Gamma, power = 3: Inverse Gaussian distribution.
        self.model = model
        self.model_des = "Generalised Linear Regression Model"
        self.grid_multiple = 1
        self.model_calc()
    
    def SVM_regressor(self, model=SVR()):
        # Set up SVM
        self.model = model
        self.model_des = "SVM Regression Model"
        self.grid_multiple = 1
        self.model_calc()
        
    def KNN_regressor(self, model=KNeighborsRegressor()):
        # Set up KNN 
        self.model = model
        self.model_des = "KNN Regression Model"
        self.grid_multiple = 1
        self.model_calc()
        
    def knn_all_k(self, limit = 50):
        # run KNN for all possible Ks and graph them
        self.scores = []
        self.max_k = np.minimum(limit,int(len(self.y)*(1-(1/self.folds))-1))
        for k in range(1, self.max_k):
            knn = KNeighborsRegressor(n_neighbors=k)
            self.scores.append(np.mean(cross_val_score(knn, self.X, self.y, cv=self.kfold)))
        self.knn_best = self.scores.index(np.max(self.scores))+1
        plt.plot(range(1, self.max_k), self.scores, label='Mean CV Scores')
        plt.hlines(self.baseline, 1, self.max_k, label='baseline')
        plt.xlabel('k')
        plt.ylabel('R2')
        plt.legend(loc=[1.1, 0])
        print(self.BOLD + "Highest KNN Score:" + self.END, self.knn_best)
        plt.show()
        
    def Decision_tree_model(self, print_tree=False, print_depth=5, model=DecisionTreeRegressor(random_state=66)):
        # set up decision tree model
        self.model = model
        self.model_des = "Decision Tree Model"
        self.grid_multiple = 1
        self.model_calc()
        if print_tree == True:
            dot_data = StringIO() 
            export_graphviz(self.model, out_file=dot_data, filled=True, rounded=True,
                            special_characters=True, feature_names=self.X.columns, max_depth=print_depth)  

            graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
            display(Image(graph.create_png()))

    def Random_forest_model(self, model=RandomForestRegressor(random_state=66)):
        self.model = model
        self.model_des = "Random Forest Model"
        self.grid_multiple = 1
        self.model_calc()
            
    def GradientBoosting(self, grad_model=GradientBoostingRegressor(random_state=66)):
        self.model = grad_model
        self.model_des = "Gradient Boosting Model"
        self.grid_multiple = 32.4
        self.model_calc()
    
    def VotingRegressor(self, model = VotingRegressor, vote_model1 = LinearRegression(), vote_model2 = RandomForestRegressor(random_state=66), vote_model3 = GradientBoostingRegressor(random_state=66)):
        reg1 = vote_model3
        reg2 = vote_model2
        reg3 = vote_model1
        self.model = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
        self.model_des = "Voting Regressor Model (default Linear & RF & GB)"
        self.grid_multiple = 1
        self.model_calc()
    
    def MLP_Neural_Net(self, model = MLPRegressor(solver='adam', alpha=10**(0),
                                        hidden_layer_sizes=(10, 10, 10), activation='relu',
                                        random_state=66, batch_size=50,max_iter=500)):
        self.model = model
        self.model_des = 'MLP Regressor Neural Net'
        self.grid_multiple = 1
        self.model_calc()

    def model_calc(self):
        # fit model
        t0 = time.time()
        self.model.fit(self.X, self.y)
        self.sc = self.model.score(self.X, self.y)
        self.cvs = cross_val_score(self.model, self.X, self.y, cv=self.kfold).mean()
        self.rmse = -cross_val_score(self.model, self.X, self.y, cv=self.kfold, scoring="neg_root_mean_squared_error").mean()
        # Get test score
        if self.test != 0:
            self.sct = self.model.score(self.X_test, self.y_test)
            self.sctp = str(round(self.sct, 4))
        else:
            self.sct = 'No Test Set'
            self.sctp = 'No Test Set'
        # time the running of the model
        t1 = time.time()
        self.elaspsed = t1-t0
        # show the results from the classification model
        if self.print_info==True:
            print(self.BOLD + self.model_des, 'Test\nTrain Score:' + self.END, round(self.sc, 4),
                  self.BOLD + '\nCV Fold Score (r^2):' +
                  self.END, round(self.cvs, 4),
                  self.BOLD + '\nCV Fold Score (RMSE):' +
                  self.END, round(self.rmse, 4),
                  self.BOLD + "\nModel Test Score:" + self.END, self.sctp)
            print("Time Elapsed = ", round(self.elaspsed, 2), 'secs - grid will take ~',
                  round(self.elaspsed*self.grid_multiple, 2), 'minutes to run.\n')
        try:
            self.coef = self.model.coef_
            self.coefs
        except:
            pass
        if self.save_it == True:
            self.tracking()
            
    def tracking(self):
        global model_tracker
        df_temp = pd.DataFrame({'model_type':self.model_des,'model_train_score':self.sc,
                                'cv_score_r2':self.cvs,'cv_score_rmse': self.rmse,'test_score':self.sct,'predictors': str(','.join(self.X.columns)),
                                'model_params':str(self.model),'observations':len(self.X),'time':self.elaspsed,'comment':self.comment},index=[1])
        try:
            model_tracker =  pd.concat([model_tracker,df_temp])
        except:
            model_tracker = pd.DataFrame(columns = ['model_type','model_train_score','cv_score_r2','cv_score_rmse',
                                                    'test_score','predictors','model_params','observations','time',
                                                    'comment'])
            model_tracker =  pd.concat([model_tracker,df_temp])

    def gridsearch(self, params='default'):
        """A function which automatically runs a gridsearch on your selected model. Returns model_grid model with best parameters.
        Has default parameters for each model type, but you can set your own by passing a dict into params = {}
        """
        # setting the default parameters if not set by user
        if params == 'default':
            if self.model_des == "Logistic Regression Model":
                self.params = {'penalty': ['l1', 'l2', 'elasticnet'], 'solver': ['saga'], 'C': np.logspace(-5, 5, 5), 'l1_ratio': np.linspace(0.0001, 1, 4)}
            elif self.model_des == "K Neighbors Model":
                self.params = {'n_neighbors': range(1, 20, 1), 'weights': ['uniform', 'distance'], 'p': [1, 2]}
            elif self.model_des == "Decision Tree Model":
                self.params = {'criterion': ['gini', 'entropy'], 'max_depth': [None, 5, 6, 7, 8], 'max_features': ['auto'], 'splitter': [
                    'best', 'random'], 'min_samples_split': [2, 3, 4, 5], 'ccp_alpha': [0.0, 0.0001, 0.001, .01, .1, 1, 10, 100], 'class_weight': [None, 'balanced']}
            elif self.model_des == "Random Forest Model":
                self.params = {'n_estimators':[100,200,500], 'criterion':['gini','entropy'], 'max_depth':[None], 'min_samples_split':[2,6],"max_features":["auto","log2"],
                               'oob_score':[True,False],'warm_start':[True,False],'ccp_alpha':[0.0,0.5,1]}
            elif self.model_des == "ADA Boosting Model": 
                self.params = {"learning_rate": [0.05, 0.25, 0.5, 0.75, 1], 'base_estimator':[DecisionTreeClassifier(max_depth=1),DecisionTreeClassifier(max_depth=2),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=4),DecisionTreeClassifier(max_depth=5)],
                               'algorithm':['SAMME'],"n_estimators":[100,200,500,1000]}
            elif self.model_des == "Gradient Boosting Model": 
                self.params = {"learning_rate": [0.01, 0.5, 1], 'loss':['deviance', 'exponential'],'max_features':['auto','log2','sqrt'],
                               'warm_start':[True,False],"n_estimators":[100,200], 'ccp_alpha':[0.0,0.5,0.9],'max_depth':[1,3,5], 'subsample':[1.0,0.75,0.5]}
            elif self.model_des == "Naive Bayes Model":
                self.params = {"var_smoothing": [0.000001, 0.2, 0.4, 0.6, 0.8, 1]}
            elif self.model_des == "Linear Support Vectors Model": 
                self.params = {'C':np.linspace(-10,10,20),'loss':['hinge','squared_hinge']}
            elif self.model_des == "Gaussian (rbf) Support Vectors Model": 
                self.params = {'C':np.linspace(-10,10,15),'gamma':np.linspace(0.00001,100,15),'kernel':['rbf']}
            elif self.model_des == "Polynomial Support Vectors Model": 
                self.params = {'C':np.linspace(-10,10,15),'coef0':[0,1,2,3,4,10],'kernel':['poly'],'degree':[2,3,4]}
            elif self.model_des == "MLP Classifier Neural Net": 
                self.params = {'solver':['adam','sgd'], 'alpha': np.linspace(0.00001,1,4),'hidden_layer_sizes':[(10, 10, 10,10),(20, 20, 20),(50,50),(100)],
                               'learning_rate' : ['constant', 'invscaling', 'adaptive'],'activation' : ['identity', 'logistic', 'tanh', 'relu']}
        else:
            self.params = params
        # setup the gridsearch
        self.grid = GridSearchCV(self.model, self.params, verbose=1, cv=StratifiedKFold(
            self.folds, shuffle=self.shuffle, random_state=66))
        self.grid.fit(self.X, self.y)
        self.gsc = self.grid.best_score_
        self.best = self.grid.best_params_
        self.model = self.grid.best_estimator_
        self.model_des = self.model_des + " Grid Search:"
        try:
            self.coef = self.grid.best_estimator_.coef_
        except:
            pass
        # Check test score for grid
        try:
            self.sct = self.grid.best_estimator_.score(
                self.X_test, self.y_test)
            self.sctp = str(round(self.sct, 4))
        except:
            self.sctp = None
        # Print Grid results
        if self.print_info==True:
            print(self.BOLD + self.model_des + self.END)
            print(self.BOLD + "Best Mean CV Model Score:" + self.END, round(self.gsc, 4),
                  self.BOLD + "\nModel Test Score:" + self.END, self.sctp)
            print(self.BOLD + 'Grid Best Parameters:\n' + self.END, self.best)
            print(self.BOLD + '\nSearch Parameters:\n' + self.END, self.params)
        self.coefs()

In [16]:
df = pd.read_csv("train_amended.csv")
model_tracker = pd.read_csv('model_tracker.csv')
y = df.pop("y")
X = df.iloc[:,2:]

In [17]:
model_tracker

model_type  model_train_score  \
0                            Linear Regression Model           0.900824   
1         Linear Regression Model (with Elastic Net)           0.000000   
2                Generalised Linear Regression Model           0.844314   
3                               SVM Regression Model           0.203636   
4                               KNN Regression Model           0.933828   
5                                Decision Tree Model           1.000000   
6                                Random Forest Model           0.982509   
7                            Gradient Boosting Model           0.997897   
8  Voting Regressor Model (default Linear & RF & GB)           0.978455   
9                           MLP Regressor Neural Net           0.738449   

   cv_score_r2  cv_score_rmse   test_score  \
0     0.752000       0.041951  No Test Set   
1    -0.003083       0.086114  No Test Set   
2     0.817128       0.036760  No Test Set   
3     0.086303       0.081352  No Test Set   
4     0.876282       0.030163  No Test Set   
5     0.740263       0.043357  No Test Set   
6     0.869160       0.031106  No Test Set   
7     0.867004       0.031362  No Test Set   
8     0.866496       0.031350  No Test Set   
9     0.706093       0.046755  No Test Set   

                                          predictors  \
0  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
2  existence expectancy index,existence expectanc...   
3  existence expectancy index,existence expectanc...   
4  existence expectancy index,existence expectanc...   
5  existence expectancy index,existence expectanc...   
6  existence expectancy index,existence expectanc...   
7  existence expectancy index,existence expectanc...   
8  existence expectancy index,existence expectanc...   
9  existence expectancy index,existence expectanc...   

                                        model_params  observations      time  \
0                                 LinearRegression()           334  0.082119   
1                                       ElasticNet()           334  0.043729   
2                                 TweedieRegressor()           334  0.055200   
3                                              SVR()           334  0.052393   
4                              KNeighborsRegressor()           334  0.086771   
5             DecisionTreeRegressor(random_state=66)           334  0.096885   
6             RandomForestRegressor(random_state=66)           334  4.083646   
7         GradientBoostingRegressor(random_state=66)           334  2.681762   
8  VotingRegressor(estimators=[('gb', GradientBoo...           334  7.255902   
9  MLPRegressor(alpha=1, batch_size=50, hidden_la...           334  7.084805   

                                      comment  
0  First run with no nulls, no galaxy or year  
1  First run with no nulls, no galaxy or year  
2  First run with no nulls, no galaxy or year  
3  First run with no nulls, no galaxy or year  
4  First run with no nulls, no galaxy or year  
5  First run with no nulls, no galaxy or year  
6  First run with no nulls, no galaxy or year  
7  First run with no nulls, no galaxy or year  
8  First run with no nulls, no galaxy or year  
9  First run with no nulls, no galaxy or year

In [18]:
X

existence expectancy index  existence expectancy at birth  \
0                       0.628657                      63.125200   
1                       0.818082                      81.004994   
2                       0.659443                      59.570534   
3                       0.555862                      52.333293   
4                       0.991196                      81.802464   
...                          ...                            ...   
3860                    1.029704                      82.832063   
3861                    0.937869                      75.877098   
3862                    1.036144                      93.540275   
3863                    0.939034                      78.274427   
3864                    1.032244                      91.641356   

      Gross income per capita  Income Index  \
0                27109.234310      0.646039   
1                30166.793958      0.852246   
2                 8441.707353      0.499762   
3                31633.240872      0.825154   
4                81033.956906      1.131163   
...                       ...           ...   
3860             34310.471408      0.855094   
3861             36899.067719      0.929494   
3862             37002.977875      1.085245   
3863             28180.459770      0.687655   
3864             73109.215949      1.207746   

      Expected years of education (galactic years)  \
0                                         8.240543   
1                                        10.671823   
2                                         8.840316   
3                                        14.723296   
4                                        13.800672   
...                                            ...   
3860                                     18.578586   
3861                                     16.153857   
3862                                     21.066473   
3863                                      9.388911   
3864                                     18.910920   

      Mean years of education (galactic years)  \
0                                    10.283959   
1                                     4.742470   
2                                     5.583973   
3                                    10.283959   
4                                    13.188907   
...                                        ...   
3860                                 10.557143   
3861                                  9.151665   
3862                                 16.661344   
3863                                  8.908748   
3864                                 16.202486   

      Intergalactic Development Index (IDI)  Education Index  \
0                                  0.804246         0.745900   
1                                  0.833624         0.467873   
2                                  0.469110         0.363837   
3                                  0.804246         0.745900   
4                                  0.910341         0.918353   
...                                     ...              ...   
3860                               0.906573         0.862826   
3861                               0.865822         0.747577   
3862                               0.983835         1.100779   
3863                               0.735694         0.602703   
3864                               1.171634         1.085080   

      Intergalactic Development Index (IDI), Rank  \
0                                      135.129178   
1                                      152.522198   
2                                      209.813266   
3                                      135.129178   
4                                       71.885345   
...                                           ...   
3860                                   144.896214   
3861                                   164.692000   
3862                                    63.726437   
3863                                   216.805701   
3864                              

In [19]:
reg_model = full_regression(X, y, run_all = True, standardize=True, test_size=0, folds=3, save_it=True, comment = "Second run with imputed mean, no galaxy or year")

Linear Regression Model Test
Train Score: 0.6367 
CV Fold Score (r^2): 0.6164 
CV Fold Score (RMSE): 0.0392 
Model Test Score: No Test Set
Time Elapsed =  0.18 secs - grid will take ~ 0.18 minutes to run.

Linear Regression Model (with Elastic Net) Test
Train Score: 0.0 
CV Fold Score (r^2): -0.0009 
CV Fold Score (RMSE): 0.0634 
Model Test Score: No Test Set
Time Elapsed =  0.07 secs - grid will take ~ 0.07 minutes to run.

Generalised Linear Regression Model Test
Train Score: 0.587 
CV Fold Score (r^2): 0.581 
CV Fold Score (RMSE): 0.041 
Model Test Score: No Test Set
Time Elapsed =  0.08 secs - grid will take ~ 0.08 minutes to run.

SVM Regression Model Test
Train Score: 0.1969 
CV Fold Score (r^2): 0.1721 
CV Fold Score (RMSE): 0.0574 
Model Test Score: No Test Set
Time Elapsed =  0.78 secs - grid will take ~ 0.78 minutes to run.

KNN Regression Model Test
Train Score: 0.8946 
CV Fold Score (r^2): 0.8152 
CV Fold Score (RMSE): 0.0272 
Model Test Score: No Test Set
Time Elapsed =  3

In [22]:
reg_model.VotingRegressor(vote_model1=KNeighborsRegressor())

Voting Regressor Model (default Linear & RF & GB) Test
Train Score: 0.9386 
CV Fold Score (r^2): 0.8196 
CV Fold Score (RMSE): 0.0269 
Model Test Score: No Test Set
Time Elapsed =  57.01 secs - grid will take ~ 57.01 minutes to run.



In [24]:
model_tracker.sort_values('cv_score_rmse')

model_type  model_train_score  \
1  Voting Regressor Model (default Linear & RF & GB)           0.938610   
1                               KNN Regression Model           0.894570   
1                                Random Forest Model           0.972292   
1                            Gradient Boosting Model           0.893800   
1  Voting Regressor Model (default Linear & RF & GB)           0.891760   
4                               KNN Regression Model           0.933828   
6                                Random Forest Model           0.982509   
8  Voting Regressor Model (default Linear & RF & GB)           0.978455   
7                            Gradient Boosting Model           0.997897   
2                Generalised Linear Regression Model           0.844314   
1                            Linear Regression Model           0.636701   
1                Generalised Linear Regression Model           0.586988   
0                            Linear Regression Model           0.900824   
1                                Decision Tree Model           1.000000   
5                                Decision Tree Model           1.000000   
9                           MLP Regressor Neural Net           0.738449   
1                               SVM Regression Model           0.196850   
1         Linear Regression Model (with Elastic Net)           0.000000   
1                           MLP Regressor Neural Net          -0.000025   
3                               SVM Regression Model           0.203636   
1         Linear Regression Model (with Elastic Net)           0.000000   

   cv_score_r2  cv_score_rmse   test_score  \
1     0.819558       0.026909  No Test Set   
1     0.815157       0.027242  No Test Set   
1     0.787532       0.029194  No Test Set   
1     0.783238       0.029476  No Test Set   
1     0.777133       0.029903  No Test Set   
4     0.876282       0.030163  No Test Set   
6     0.869160       0.031106  No Test Set   
8     0.866496       0.031350  No Test Set   
7     0.867004       0.031362  No Test Set   
2     0.817128       0.036760  No Test Set   
1     0.616364       0.039219  No Test Set   
1     0.581031       0.041009  No Test Set   
0     0.752000       0.041951  No Test Set   
1     0.547820       0.042363  No Test Set   
5     0.740263       0.043357  No Test Set   
9     0.706093       0.046755  No Test Set   
1     0.172118       0.057379  No Test Set   
1    -0.000857       0.063355  No Test Set   
1    -0.006909       0.063539  No Test Set   
3     0.086303       0.081352  No Test Set   
1    -0.003083       0.086114  No Test Set   

                                          predictors  \
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
4  existence expectancy index,existence expectanc...   
6  existence expectancy index,existence expectanc...   
8  existence expectancy index,existence expectanc...   
7  existence expectancy index,existence expectanc...   
2  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
0  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
5  existence expectancy index,existence expectanc...   
9  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   
3  existence expectancy index,existence expectanc...   
1  existence expectancy index,existence expectanc...   

                                        model_params  observations       time  \
1  VotingRegressor(estimators=[('gb', GradientBoo...          38

In [21]:
model_tracker.to_csv("model_tracker.csv", index=False)